In [5]:
from hashlib import md5

import numpy as np
import pandas as pd
from ase.io.lammpsdata import read_lammps_data


def compute_hash(row):

    content = row.to_json()
    return md5(content.encode()).hexdigest()


def id_from_hash(hsh, char):

    return f"{hsh[0:char]!s}"


def compute_kinetic(energy, mass, dist, theta, phi):

    z = dist
    r = z / np.cos(np.radians(theta))
    x = (r * np.sin(np.radians(theta)) * np.cos(np.radians(phi)))
    y = (r * np.sin(np.radians(theta)) * np.sin(np.radians(phi)))
        
    # use scipy atomic mass unit-electron volt relationship and speed of light
    v = np.sqrt(2 * energy / mass / 931494102.42) * 299792458.0 * 1e-2

    vx = v * np.sin(np.pi - np.radians(theta)) * np.cos(np.pi + np.radians(phi))
    vy = v * np.sin(np.pi - np.radians(theta)) * np.sin(np.pi + np.radians(phi))
    vz = v * np.cos(np.pi - np.radians(theta))

    return x, y, z, v, vx, vy, vz


# load data from config file into numpy structures
ion_data = {
    "theta": 0,
    "phi": np.empty(9000),
}

# build dataframe with combinations of theta and number of phi samples
db = pd.DataFrame(
    np.array(np.meshgrid(*ion_data.values())).T.reshape(-1, len(ion_data.keys())),
    columns=ion_data.keys(),
    dtype=float,
)

# fill in values of phi from uniform distribution
rng = np.random.default_rng(seed=190820222)
db["phi"] = rng.uniform(
    0, 180, size=len(db)
)

# set unique simulation IDs
hashes = db[ion_data.keys()].apply(compute_hash, axis=1)
id_chars = 7
db["run_id"] = range(1, len(hashes) + 1)

energy=200

# fill in values of position and velocity, given theta and phi
db[["x0", "y0", "z", "v", "vx", "vy", "vz"]] = db.apply(
    lambda x: compute_kinetic(
        energy=200,
        mass=2.014,
        dist=8,
        theta=x.theta,
        phi=x.phi,
    ),
    axis=1,
    result_type="expand",
)

# load minimal cell
with open('/u/49/fakhran1/unix/Desktop/Docs/Simulations/Be/Main/1. Relaxation/300K/D/BeD/BeD.openZ.eq_300.data', "r") as f:
    mini = read_lammps_data(f, style="atomic")

# sample uniformly in minimal cell size and shift ion position
lx, ly, lz = mini.cell.lengths()

db["shift_x"] = rng.uniform(-0.5, 0.5, size=len(db))
db["shift_y"] = rng.uniform(-0.5, 0.5, size=len(db))

# clean up and output dataframe
db.sort_values(by=["theta", "run_id"], inplace=True)
db.set_index("run_id", inplace=True)
comments = {
    "run id": "-",
    "ion theta": "deg",
    "ion phi": "deg",
    "initial x position": "Å",
    "initial y position": "Å",
    "initial/final z position": "Å",
    "initial velocity": "Å/ps",
    "initial x velocity": "Å/ps",
    "initial y velocity": "Å/ps",
    "initial z velocity": "Å/ps",
}
with open(f'/u/49/fakhran1/unix/Desktop/Input_{energy}eV.tsv', "w") as f:
    f.write("# " + "\t".join([k for k, v in comments.items()]) + "\n")
    f.write("# " + "\t".join([v for k, v in comments.items()]) + "\n")
    db.to_csv(f, sep="\t", float_format="%.9f")
with pd.option_context("display.max_rows", 50):
    print(db)
print(f"There are {len(db.index)} simulations in this set")

        theta         phi   x0   y0    z            v            vx  \
run_id                                                                
1         0.0  140.316850 -0.0  0.0  8.0  1384.302205  1.304667e-13   
2         0.0   63.716913  0.0  0.0  8.0  1384.302205 -7.506816e-14   
3         0.0  168.823810 -0.0  0.0  8.0  1384.302205  1.663132e-13   
4         0.0   77.072494  0.0  0.0  8.0  1384.302205 -3.792650e-14   
5         0.0  131.000433 -0.0  0.0  8.0  1384.302205  1.112214e-13   
...       ...         ...  ...  ...  ...          ...           ...   
8996      0.0  109.383852 -0.0  0.0  8.0  1384.302205  5.626558e-14   
8997      0.0   78.290169  0.0  0.0  8.0  1384.302205 -3.440663e-14   
8998      0.0   34.943715  0.0  0.0  8.0  1384.302205 -1.389648e-13   
8999      0.0   31.165552  0.0  0.0  8.0  1384.302205 -1.450611e-13   
9000      0.0   36.662103  0.0  0.0  8.0  1384.302205 -1.359905e-13   

                  vy           vz   shift_x   shift_y  
run_id              